In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

# 1. 데이터 로드 및 전처리

In [2]:
df = pd.read_csv('./data/framingham.csv')

In [4]:
# 분석에 사용할 변수들
cols = ['currentSmoker', 'age', 'male', 'BMI', 'sysBP', 'totChol', 'glucose', 'TenYearCHD']
df = df.dropna(subset=cols)

In [6]:
# treatment: currentSmoker (1: 흡연, 0: 비흡연)
T = df['currentSmoker']
# outcome: 10년 내 심장질환 발생 (1: 발생, 0: 미발생)
Y = df['TenYearCHD']
# confounders: 나이, 성별, BMI, 수축기 혈압, 총콜레스테롤, 혈당
X = df[['age', 'male', 'BMI', 'sysBP', 'totChol', 'glucose']]

# 2. Propensity Score 계산 (로지스틱 회귀 사용)

In [7]:
logit = LogisticRegression(max_iter=1000)
logit.fit(X, T)
df['propensity'] = logit.predict_proba(X)[:, 1]

C:\Users\PC\AppData\Local\Temp\ipykernel_23236\3840394917.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['propensity'] = logit.predict_proba(X)[:, 1]


# 3. Nearest Neighbor Matching (1:1, 대체 없이)

In [10]:
treated = df[df['currentSmoker'] == 1]
control = df[df['currentSmoker'] == 0]

matched_pairs = []
control_indices_used = set()

for treated_idx, treated_row in treated.iterrows():
    ps_treated = treated_row['propensity']
    # 아직 매칭되지 않은 control group 추출
    available_controls = control.loc[~control.index.isin(control_indices_used)]
    if available_controls.empty:
        break  # 더 이상 매칭 가능한 control이 없으면 종료
    # 흡연군과 가장 가까운 propensity score을 가진 대조군 찾기
    closest_control_idx = (available_controls['propensity'] - ps_treated).abs().idxmin()
    matched_pairs.append((treated_idx, closest_control_idx))
    control_indices_used.add(closest_control_idx)

In [11]:
# 매칭된 인덱스 추출
matched_treated_indices = [pair[0] for pair in matched_pairs]
matched_control_indices = [pair[1] for pair in matched_pairs]

# 4. 인과 효과 추청: 매칭된 샘플에서 outcome의 평균 차이 계산

In [12]:
treated_outcomes = df.loc[matched_treated_indices, 'TenYearCHD']
control_outcomes = df.loc[matched_control_indices, 'TenYearCHD']

effect = treated_outcomes.mean() - control_outcomes.mean()
print('Estimated Treatment Effect (Difference in 10-year CHD rates):', effect)

Estimated Treatment Effect (Difference in 10-year CHD rates): 0.02182011708355508


# 결과 해석
- 이 결과는 매칭된 샘플에서 **흡연군이 비흡연군보다 약 2.18% 높은 심장질환 발생률**을 보인다는 것을 의미합니다.
- 즉, 통제된 혼란 변수들(나이, 성별, BMI, 혈압, 콜레스테롤, 혈당)을 고려할 때, 흡연이 10년 내 심장질환 발생 위험을 증가시키는 요인으로 작용할 가능성이 있음을 시사합니다.
- 다만, 이 결과는 관찰 연구 기반의 추정치이므로, 잠재적 교란변수나 매칭의 질, 모형의 가정 등에 대한 추가 검증이 필요합니다.